In [1]:
from google.colab import drive
drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
datapath="/content/drive/MyDrive/Colab Notebooks/NLP/fra-eng/fra.txt"
datapathhe="/content/drive/MyDrive/Colab Notebooks/NLP/hin-eng/hin.txt"

In [3]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,LSTM,Dense
import numpy as np
epochs=100
batch_size=64
latent_dim=256
num_samples=100000

In [4]:
input_texts=[]
target_texts=[]
input_character=set()
target_character=set()
with open(datapath, 'r',encoding='utf-8') as f:
  lines=f.read().split('\n');
for line in lines[:min(num_samples,len(lines)-1)]:
  input_text,target_text,_=line.split('\t')
  target_text='\t'+target_text+'\n'
  input_texts.append(input_text)
  target_texts.append(target_text)
  for char in input_text:
    if char not in input_character:
      input_character.add(char)
  for char in target_text:
    if char not in target_character:
      target_character.add(char)

In [5]:
input_character=sorted(list(input_character))
target_character=sorted(list(target_character))
num_encoder_tokens=len(input_character)
num_decoder_tokens=len(target_character)
max_encoder_seq_length=max([len(txt) for txt in input_texts])
max_decoder_seq_length=max([len(txt) for txt in target_texts])

In [6]:
print("input_character: ",input_character)
print("target_character:",target_character)
print("num_encoder_token:",num_encoder_tokens)
print("num_decoder_token:",num_decoder_tokens)
print("max_encoder_seq_length:",max_encoder_seq_length)
print("max_decoder_seq_length:",max_decoder_seq_length)

input_character:  [' ', '!', '"', '$', '%', '&', "'", ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'é', '’', '€']
target_character: ['\t', '\n', ' ', '!', '"', '$', '%', '&', "'", '(', ')', '+', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\xa0', '«', '»', 'À', 'Ç', 'É', 'Ê', 'Ô', 'à', 'â', 'ç', 'è', 'é', 'ê', 'ë', 'î', 'ï', 'ô', 'ù', 'û', 'œ', 'С', '\u2009', '\u200b', '‘', '’', '\u202f']
num_encoder_token: 78


In [7]:
input_token_index=dict([(char,i) for i,char in enumerate(input_character)])
target_token_index=dict([(char,i) for i,char in enumerate(target_character)])

In [8]:
encoder_input_data=np.zeros((len(input_texts),max_encoder_seq_length,num_encoder_tokens),dtype="float32")
decoder_input_data=np.zeros((len(input_texts),max_decoder_seq_length,num_decoder_tokens),dtype="float32")

In [9]:
decoder_target_data=np.zeros((len(input_texts),max_decoder_seq_length,num_decoder_tokens),dtype="float32")

In [10]:
for i,(input_text,target_text) in enumerate(zip(input_texts,target_texts)):
  for t,char in enumerate(input_text):
    encoder_input_data[i,t,input_token_index[char]]=1
  encoder_input_data[i,t+1:,input_token_index[' ']]=1

  for t,char in enumerate(target_text):
    decoder_input_data[i,t,target_token_index[char]]=1
    if t>0:
      decoder_target_data[i,t-1,target_token_index[char]]=1
  decoder_input_data[i,t+1:,target_token_index[' ']]=1
  decoder_target_data[i,t:,target_token_index[' ']]=1

In [11]:
encoder_input_data[0].shape

(29, 78)

In [12]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the 
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [13]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Epoch 1/100
1250/1250 [==============================] - 28s 16ms/step - loss: 0.7097 - accuracy: 0.8006 - val_loss: 0.7024 - val_accuracy: 0.7900
Epoch 2/100
1250/1250 [==============================] - 20s 16ms/step - loss: 0.4535 - accuracy: 0.8640 - val_loss: 0.5545 - val_accuracy: 0.8336
Epoch 3/100
1250/1250 [==============================] - 19s 15ms/step - loss: 0.3783 - accuracy: 0.8859 - val_loss: 0.4885 - val_accuracy: 0.8526
Epoch 4/100
1250/1250 [==============================] - 19s 15ms/step - loss: 0.3364 - accuracy: 0.8981 - val_loss: 0.4549 - val_accuracy: 0.8626
Epoch 5/100
1250/1250 [==============================] - 20s 16ms/step - loss: 0.3104 - accuracy: 0.9055 - val_loss: 0.4318 - val_accuracy: 0.8692
Epoch 6/100
1250/1250 [==============================] - 20s 16ms/step - loss: 0.2921 - accuracy: 0.9108 - val_loss: 0.4169 - val_accuracy: 0.8734
Epoch 7/100
1250/1250 [==============================] - 20s 16ms/step - loss: 0.2785 - accuracy: 0.9147 - val_loss: 0

In [14]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

reverse_input_char_index=dict((i,char) for char,i in input_token_index.items())
reverse_target_char_index=dict((i,char) for char,i in target_token_index.items())
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


In [16]:
for seq_index in range (1000,1100):
  input_seq=encoder_input_data[seq_index:seq_index+1]
  decode_sentence=decode_sequence(input_seq)
  print('-')
  print('Input sentence:',input_texts[seq_index])
  print('target sentence:',target_texts[seq_index])
  print('decoded sentences',decode_sentence)

-
Input sentence: I crashed.
target sentence: 	Je me suis écrasé.

decoded sentences Je me suis fait tous les deux.

-
Input sentence: I crashed.
target sentence: 	Je me suis écrasée.

decoded sentences Je me suis fait tous les deux.

-
Input sentence: I cringed.
target sentence: 	J'eus un mouvement de recul.

decoded sentences J'ai réparé les yeux.

-
Input sentence: I cringed.
target sentence: 	J'ai eu un mouvement de recul.

decoded sentences J'ai réparé les yeux.

-
Input sentence: I cringed.
target sentence: 	Je suis rentré en moi-même.

decoded sentences J'ai réparé les yeux.

-
Input sentence: I exhaled.
target sentence: 	J’ai expiré.

decoded sentences J'ai débarrassé.

-
Input sentence: I frowned.
target sentence: 	J'ai froncé les sourcils.

decoded sentences J'ai froidé la question.

-
Input sentence: I gave up.
target sentence: 	J'ai abandonné.

decoded sentences J'ai abandonné.

-
Input sentence: I give in.
target sentence: 	Je donne ma langue au chat.

decoded sentences J'